In [ ]:
!pip install hep_ml

In [ ]:
from scipy.stats import chi2, expon, weibull_min
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer
from sklearn.model_selection import train_test_split
from copy import deepcopy
import os
from hep_ml import reweight
import pickle

In [ ]:
nevs = 2000000
dof = 3
############################################ sample 1 #########################################################
pdf1 = chi2(dof)
# sample the chi2 distribution
sample1 = pdf1.rvs(nevs)
# set the lowest entry to 0 and the highest to 1, then scale all the other values in between
sample1 = MinMaxScaler((0, 1)).fit_transform(sample1.reshape(-1, 1)).reshape(-1)
# create the fake targed (aka the data) with some introduced noise
print(f"shape of sample1: {sample1.shape}")
sample1_target = sample1 + 0.1 + 0.01 * np.random.normal(size=sample1.shape)
# split target1 sample into training and test samples
sample1_train, sample1_test = train_test_split(sample1, test_size=0.3)
# split the target data into training and test samples
sample1_target_train, sample1_target_test = train_test_split(sample1_target, test_size=0.3)

# plot the training sample of the MC (sample1) and data (sample1_target)
plt.hist(sample1_train, bins=100, label="MC")
plt.hist(sample1_target_train, bins=100, alpha=0.5, label="data")
plt.xlabel('a')
plt.legend()
plt.show()

############################################ sample 2 ###########################################################
pdf2 = weibull_min(10, 1)
sample2 = pdf2.rvs(nevs)
# !! Introducing some correlation by using sample1 to define sample2
sample2 = sample2 - 5*sample1
sample2 = MinMaxScaler((0, 1)).fit_transform(sample2.reshape(-1, 1)).reshape(-1)
sample2_target = sample2 - 0.05 + 0.01 * np.random.normal(size=sample2.shape)
sample2_train, sample2_test = train_test_split(sample2, test_size=0.3)
sample2_target_train, sample2_target_test = train_test_split(sample2_target, test_size=0.3)
plt.hist(sample2_train, bins=100, label="MC")
plt.hist(sample2_target_train, bins=100, alpha=0.5, label="data")
plt.legend()
plt.xlabel('b')
plt.show()

############################################ sample 3 ###########################################################
# !! Introducing correlation by using sample1 and sample2 to define sample3
sample3 = np.log1p(sample1) + np.log1p(sample2)
sample3 = MinMaxScaler((0, 1)).fit_transform(sample3.reshape(-1, 1)).reshape(-1)
sample3_target = np.log1p(sample1_target) + np.log1p(sample2_target)
sample3_target = QuantileTransformer(output_distribution='normal').fit_transform(sample3_target.reshape(-1, 1)).reshape(-1)
sample3_target = MinMaxScaler((0, 1)).fit_transform(sample3_target.reshape(-1, 1)).reshape(-1)
sample3_train, sample3_test = train_test_split(sample3, test_size=0.3)
sample3_target_train, sample3_target_test = train_test_split(sample3_target, test_size=0.3)
plt.hist(sample3_train, bins=100, label="MC")
plt.hist(sample3_target_train, bins=100, alpha=0.5, label="data")
plt.xlabel('x')
plt.legend()
plt.show()

############################################ sample 4 ###########################################################
# !! Introducing correlation by using sample1 and sample2 to define sample4
sample4 = np.log1p(sample1) + np.log1p(sample2) - np.log1p(sample1 * sample2) + 0.05
sample4 = MinMaxScaler((0, 1)).fit_transform(sample4.reshape(-1, 1)).reshape(-1)
sample4_target = np.log1p(sample1_target) + np.log1p(sample2_target) - np.log1p(sample1_target * sample2_target) + 0.05
sample4_target = QuantileTransformer(output_distribution='normal').fit_transform(sample4_target.reshape(-1, 1)).reshape(-1)
sample4_target = MinMaxScaler((0, 1)).fit_transform(sample4_target.reshape(-1, 1)).reshape(-1)
sample4_train, sample4_test = train_test_split(sample4, test_size=0.3)
sample4_target_train, sample4_target_test = train_test_split(sample4_target, test_size=0.3)
plt.hist(sample4_train, bins=100,label="MC")
plt.hist(sample4_target_train, bins=100, alpha=0.5, label="data")
plt.xlabel('y')
plt.legend()
plt.show()

############################################ store samples in df ##################################################
df = pd.DataFrame({'a': sample1_train, 'b': sample2_train, 'x': sample3_train, 'y': sample4_train})
df_target = pd.DataFrame({'a': sample1_target_train, 'b': sample2_target_train, 'x': sample3_target_train, 'y': sample4_target_train})

df_test = pd.DataFrame({'a': sample1_test, 'b': sample2_test, 'x': sample3_test, 'y': sample4_test})
df_target_test = pd.DataFrame({'a': sample1_target_test, 'b': sample2_target_test, 'x': sample3_target_test, 'y': sample4_target_test})

In [ ]:
# In analogy to our main problem: context vars are 4 Kinematic variables
context_vars = ['a', 'b']
# In analogy to our main problem: target vars are 6 Shower Shapes and 3 Isolation variables
target_vars = ['x', 'y']

reweighter = reweight.GBReweighter(
    n_estimators=250,
    learning_rate=0.1,
    max_depth=3,
    min_samples_leaf=1000,
    gb_args={"subsample": 0.4},
)

bins_reweighter = reweight.BinsReweighter(n_bins=300, n_neighs=1.)

bins_reweighter.fit(
    df[context_vars].values,
    df_target[context_vars].values,
)

train_weights = bins_reweighter.predict_weights(
    df[context_vars].values,
)

test_weights = bins_reweighter.predict_weights(
    df_test[context_vars].values,
)

# add weights to mc dataframe and ones to data dataframe
df["weight"] = train_weights
df_target["weight"] = np.ones(len(df_target))
df_test["weight"] = test_weights
df_target_test["weight"] = np.ones(len(df_target_test))

# plot
for var in context_vars + target_vars:
    plt.hist(df_test[var], bins=100, density=True, weights=df_test['weight'])
    plt.hist(df_target_test[var], bins=100, alpha=0.5, density=True)
    plt.xlabel(var)
    plt.show()

In [ ]:
output_dir = "./samples"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
pickle.dump(bins_reweighter, open('./samples/reweighter.pkl', "wb"))

# save as parquet the four samples  
df.to_parquet(os.path.join(output_dir, "train.parquet"))    
df_target.to_parquet(os.path.join(output_dir, "train_target.parquet"))
df_test.to_parquet(os.path.join(output_dir, "test.parquet"))
df_target_test.to_parquet(os.path.join(output_dir, "test_target.parquet"))